# **DSS Partial Order Resolution Probabilsitic Models**
---
---

In [1]:
# imports
from pprint import pprint
from itertools import chain, permutations, product
from tqdm import tqdm
import pm4py
import datetime

### Helper Functions

In [2]:
# event set functions
def pos_res_of_event_set(event_set: list) -> list:
    """For a given events set, returns the list of all possible resolutions."""
    
    return [list(tup) for tup in permutations(event_set, len(event_set))]


def pos_res_for_unc_trace(unc_trace: list) -> list:
    """Return all the possible resolution for an uncertain trace."""
        
    resolutions_for_event_sets = [pos_res_of_event_set(unc_set) for unc_set in unc_trace]  # builds the pos_res for each event set in the trace, e.g. {C,B} -> [C,B] and [B,C]
    
    all_pos_res  = list(product(*resolutions_for_event_sets))  # builds all the pos res, i.e. all combinations of all the pos res for the event sets,
                                                               # e.g. [[A], [[C,B], [B,C]], [D]] -> [A,B,C,D] and [A,C,B,D]

    return [list(chain(*res)) for res in all_pos_res]  # make each pos res just a list of activities


# time stamp functions
def remove_timezones(log):
    """ Takes the timezone offset of each timestamp and adds it to the timestamp + removes the timezone."""
    
    for trace in log:
        for event in trace:
            tz_offset = event["time:timestamp"].tzinfo.utcoffset(event["time:timestamp"])
            event["time:timestamp"] = event["time:timestamp"].replace(tzinfo=None) + tz_offset
    return log

def abstract_time(log, time_func):
    """ Abstract the specified time level (in time_func) from the timestamps of the log.
        Possible: time_func <-- abstract_{microseconds, seconds, minutes, hours, 
                                          day, month, year}.
    """
    for trace in log:
        for event in trace:
            event["time:timestamp"] = time_func(event["time:timestamp"])

def abstract_microseconds(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(microsecond=0)

def abstract_seconds(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(second=0, microsecond=0)

def abstract_minutes(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(minute=0, second=0, microsecond=0)

def abstract_hours(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(hour=0, minute=0, second=0, microsecond=0)

def abstract_day(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(day=1, hour=0, minute=0, second=0, microsecond=0)

def abstract_month(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(month=1, day=1, hour=0, minute=0, second=0, microsecond=0)

def abstract_year(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace(year=1993, month=1, day=1, hour=0, minute=0, second=0, microsecond=0)

def copy_timestamp(timestamp: datetime.datetime) -> datetime.datetime:
    return timestamp.replace()

Load and Preprocess Log

In [3]:
# define file path'

bpic14     = "./logs/BPI_Challenge_2014.xes"
traffic    = "./logs/traffic_fines.xes"
artificial = "./logs/1561989897313-3_75.xes"

In [4]:
event_log = pm4py.read_xes(traffic)

parsing log, completed traces ::   0%|          | 0/150370 [00:00<?, ?it/s]

In [42]:
# the logs needs to be pre-processed

# remove timezones 
log = remove_timezones(event_log)

# abstract seconds for artificial log only
#abstract_time(event_log, abstract_seconds)

### Trace Equivalence Model
---

estimates the probability of a resolution by observing how often the given activity sequence is present in the certain log <br>
two resolutions are equivalent, if they consist of the same sequence of executed activities <br>
the certain log is defined as $ \Sigma_{certain} = \{ \sigma \in \Sigma \; \vert \; \vert \Phi(\sigma) \vert = 1 \}$ <br>
The probability of a resolution $\phi \in \Phi(\sigma)$ of a trace $\sigma$ is then quantified as: 
$$ P_{trace}(\phi) = \frac{\vert \{ \sigma \in \Sigma_{certain} \; \vert \; \exists \phi' \in \Phi(\sigma): \phi' \equiv \phi \} \vert}{\vert \Sigma_{certain} \vert} $$
Note that $\phi'$ in this case is the only resolution for $\sigma$, since $\sigma \in \Sigma_{certain}$ and thus $\vert \Phi(\sigma) \vert = 1$. <br>
So for a given uncertain trace, the probability measures how often a particular resolution occurs in the certain log, i.e. the traces without order uncertainty.

In [96]:
class TraceEquivalenceModel():
    
    
    def __init__(self, log):
        self.NAME = "concept:name"
        self.TIME = "time:timestamp"
        
        self.certain_log, self.uncertain_log, self.ground_truth_log = self.__split_and_sparse_log(log)
        self.certain_trace_freq = self.__get_trace_frequencies()
    
    
    def __split_and_sparse_log(self, log):
        """Extracts the certain part from a given log."""
        certain_log, uncertain_log, ground_truth_log = [], [], []
        for trace in log:
            if self.__trace_is_certain(trace):
                sparse_trace = self.__get_sparse_trace(trace)
                certain_log.append(sparse_trace)
            else:
                sparse_trace_set = self.__get_sparse_trace_set(trace)
                uncertain_log.append(sparse_trace_set)
                
                sparse_trace = self.__get_sparse_trace(trace)  # get the ground truth according to gold standard
                ground_truth_log.append(sparse_trace)          # i.e. order in the log
        
        return certain_log, uncertain_log, ground_truth_log
    
    
    def __get_sparse_trace(self, trace):
        """ Make a list of activities from the pm4py trace object. """
        sparse_trace = [event[self.NAME] for event in trace]
        return sparse_trace
    
    
    def __get_sparse_trace_set(self, trace) -> dict:
        trace_set = dict()
        for event in trace:
            trace_set[str(event[self.TIME])] = trace_set.get(str(event[self.TIME]), []) + [event[self.NAME]]
  
        trace_set = [event_set for event_set in trace_set.values()]

        return trace_set
    
    
    def __trace_is_certain(self, trace):
        """ Check if a trace is certain."""
        for i in range(len(trace)-1):
            if trace[i][self.TIME] == trace[i+1][self.TIME]:
                return False
        return True
    
    
    def __get_trace_frequencies(self):
        trace_freq = {}
        for trace in self.certain_log:
            trace_freq[tuple(trace)] = trace_freq.get(tuple(trace), 0) + 1
        return trace_freq


    def P_trace(self, pos_res: list) -> float:
        """
        Evaluates the probability of a possible resolution of a trace.

        Input: Possible resolution (list) of a trace.

        Returns: The probability for that given possible resolution.
        """
        try:
            return self.certain_trace_freq[tuple(pos_res)] / len(self.certain_log)
        except:
            #print("The certain trace does not contain this possible resolution. Probability undefined.")
            return 0
    
    
    def eval_model(self):
        correct_pred = 0
        total_pred = len(self.ground_truth_log)  # 25502
        for i in range(total_pred):
            correct_pred += self.eval_uncertain_trace(self.uncertain_log[i], self.ground_truth_log[i], i)
        
        print()
        print('Accuracy:', correct_pred / total_pred )
                       
        
    def eval_uncertain_trace(self, unc_trace, truth_trace, i):
        """ Determine the most probable possible resolution for a given uncertain trace. """
        pos_resolutions = pos_res_for_unc_trace(unc_trace)  
        probs = [(pos_res, self.P_trace(pos_res)) for pos_res in pos_resolutions]  # get tuple (prob, pos_res) for each pos_res
        del pos_resolutions  # need RAM
        probs.sort(key=lambda tup: tup[1], reverse=True)
        
        pred_trace = probs[0][0]
        prob = probs[0][1]
        del probs  # need RAM
        
        if i % 1000 == 0:
            print(i, unc_trace, (pred_trace, prob), truth_trace)
        
        return pred_trace == truth_trace

In [97]:
trace_equiv_model = TraceEquivalenceModel(event_log)

In [98]:
print(len(trace_equiv_model.certain_log))
print(len(trace_equiv_model.uncertain_log))
print(len(trace_equiv_model.ground_truth_log))

506
494
494


In [99]:
trace_equiv_model.eval_model()

0 [['tau split8'], ['a10', 'tau join9'], ['g12'], ['c13'], ['e14'], ['f15', 'd16']] (['tau split8', 'a10', 'tau join9', 'g12', 'c13', 'e14', 'f15', 'd16'], 0.003952569169960474) ['tau split8', 'a10', 'tau join9', 'g12', 'c13', 'e14', 'f15', 'd16']

Accuracy: 0.631578947368421


### N-Gram Model
---

approach based on $N-gram$ $approximation$ <br> 
given a resolution $\phi = \langle a_1, ..., a_n \rangle $ at first for each activity the respective probability to be at that position in the trace / pos res is estimated <br> 
for the pos res $\phi_1 = \langle a, b, c, d, f, g\rangle $ and $N = 4$ the likeliness of the activity sequence $\langle b, c, d \rangle$ being followed by $f$. <br>
the predicate $certain$ in this case for an activity sequence $A = \langle a_1, ..., a_m\rangle$ and a trace $\sigma = E_1, ..., E_n \rangle$ holds, iff the trace contains the respective activity sequence without order uncertainty ($m \leq n$)
$$certain(A, \sigma) \iff \exists \; i \in \{0, ..., n-m \}, \forall \; j \in \{ 1, ..., m \}: \;\; E_{i+j} = \{e_{i+j}\} \land \lambda(e_{i+j}) = a_j$$
The probability of an acitvity a to follow an activity sequence A is then defined as the fraction traces in the log where the sequence is (not) followed by a:
$$P(a \; \vert \langle a_1, ..., a_m\rangle = \frac{\vert \{ \sigma \in \Sigma \; \vert \; certain(\langle a_1, ..., a_m, a\rangle, \sigma) \} \vert}{\vert \{ \sigma \in \Sigma \; \vert \; certain(\langle a_1, ..., a_m \rangle, \sigma) \} \vert} $$ 
based thereon, the n-gram probability for a possible resolution $\phi = \langle e_1, ..., e_n \rangle$ aggregates the probabilities of all its events:
$$P_{N-gram}(\phi) = \prod_{k=2}^{n} P( \; \lambda(e_k) \; \vert \; \langle \lambda(e_{max(1, k-N+1)}), ..., \lambda(e_{k-1}) \rangle \;) $$

In [43]:
class NGramModel():

    
    def __init__(self, log):
        self.NAME = "concept:name"
        self.TIME = "time:timestamp"
        
        self.log = log
        self.log_set = self.__make_log_set()

        self.certain_sequences = [self.__make_certain_sequences(trace_set) for trace_set in self.log_set]
        
        self.uncertain_log, self.ground_truth_log = self.__split_and_sparse_log(log)

    
    def __make_log_set(self) -> list:
        log_set = []
        for trace in self.log:
            log_set.append(self.__make_trace_sets(trace))
        return log_set

    
    def __make_trace_sets(self, trace) -> dict:
        trace_set = dict()
        for event in trace:
            trace_set[str(event[self.TIME])] = trace_set.get(str(event[self.TIME]), []) + [event[self.NAME]]
        return trace_set

    
    def __make_certain_sequences(self, trace_set) -> list:
        '''
        For each uncertain trace we cut out the certain subtraces, e.g. [{1}, {2,3}, {4}, {5}] -> [[1],[4,5]]
        And in those we search for an activity sequence to be present
        Because for an activity sequence to be certain in a trace it must apper in a certain sequence of a trace
        '''
        certain_sequences = []
        certain_sequence = []
        for i, timestamp in enumerate(trace_set):
            if len(trace_set[timestamp]) == 1:
                if i == len(trace_set)-1:
                    certain_sequence.append(trace_set[timestamp][0])
                    certain_sequences.append(certain_sequence)
                else:
                    certain_sequence.append(trace_set[timestamp][0])
            else:
                if certain_sequence:
                    certain_sequences.append(certain_sequence)
                certain_sequence = []
        return certain_sequences
    
    
    def __split_and_sparse_log(self, log):
        """Prepare the data for evaluation, i.e. the uncertain log and corresponding ground truth."""
        
        uncertain_log, ground_truth_log = [], []
        for trace in log:
            if not self.__trace_is_certain(trace):
                sparse_trace_set = self.__get_sparse_trace_set(trace)
                uncertain_log.append(sparse_trace_set)

                sparse_trace = self.__get_sparse_trace(trace)  # get the ground truth according to gold standard
                ground_truth_log.append(sparse_trace)          # i.e. order in the log

        return uncertain_log, ground_truth_log
    
    
    def __get_sparse_trace(self, trace):
        """ Make a list of activities from the pm4py trace object. """
        sparse_trace = [event[self.NAME] for event in trace]
        return sparse_trace
    
    
    def __get_sparse_trace_set(self, trace) -> dict:
        trace_set = dict()
        for event in trace:
            trace_set[str(event[self.TIME])] = trace_set.get(str(event[self.TIME]), []) + [event[self.NAME]]
  
        trace_set = [event_set for event_set in trace_set.values()]

        return trace_set
    
    
    def __trace_is_certain(self, trace):
        """ Check if a trace is certain."""
        for i in range(len(trace)-1):
            if trace[i][self.TIME] == trace[i+1][self.TIME]:
                return False
        return True

    
    def __activities_in_sequence(self, activity_sequence: list, certain_sequence: list) -> bool:
        for i in range(len(certain_sequence) - len(activity_sequence) + 1):
            if certain_sequence[i:i+len(activity_sequence)] == activity_sequence:
                return True
        return False

    
    def certain(self, activity_sequence: list, trace_set: dict) -> bool:
        certain_sequences = self.__make_certain_sequences(trace_set)
        for certain_sequence in certain_sequences:
            if self.__activities_in_sequence(activity_sequence, certain_sequence):
                return True
        return False
    
    
    def P_a_activity_sequence(self, activity: str, activity_sequence: list) -> float:
        n_sequence_plus_activity_is_certain = 0
        n_sequence_is_certain = 0
        
        for trace_set in self.log_set:
            if self.certain(activity_sequence, trace_set):
                n_sequence_is_certain += 1
                if self.certain(activity_sequence + [activity], trace_set):
                    n_sequence_plus_activity_is_certain += 1
        
        if n_sequence_is_certain == 0: 
            return 0
        return n_sequence_plus_activity_is_certain/n_sequence_is_certain

    
    def P_n_gram(self, pos_res: list, n: int=2):
        #possible_resolution like [a, b, c]
        lower_bound = 1         # 2 in the paper, but indexing here starts one before
        upper_bound = len(pos_res)
        result = 1.0
        for i in range(lower_bound,upper_bound):
            s_index = max(i-n+1, 0)         #the gram is not n long in the beginning, its 2 long, then 3, ... until it's always n long
            result *= self.P_a_activity_sequence(pos_res[i], pos_res[s_index:i])
        return result
    
    
    def eval_model(self, n: int=2):
        correct_pred = 0
        total_pred = len(self.ground_truth_log)  # bpic14: 25502
        
        for i in tqdm(range(total_pred)):
            correct_pred += self.eval_uncertain_trace(self.uncertain_log[i], self.ground_truth_log[i], n, i)
        
        print()
        print('Accuracy:', correct_pred / total_pred )
                       
        
    def eval_uncertain_trace(self, unc_trace, truth_trace, n, i):
        """ Determine the most probable possible resolution for a given uncertain trace. """
        
        pos_resolutions = pos_res_for_unc_trace(unc_trace)  
        probs = [(pos_res, self.P_n_gram(pos_res, n)) for pos_res in pos_resolutions]  # get tuple (prob, pos_res) for each pos_res
        probs.sort(key=lambda tup: tup[1], reverse=True)
        
        pred_trace = probs[0][0]
        prob = probs[0][1]
        
        if i % 100 == 0:
            print(i, unc_trace, (pred_trace, prob), truth_trace)
        
        return pred_trace == truth_trace

In [44]:
n_gram_model = NGramModel(event_log)

In [45]:
print(len(n_gram_model.log))
print(len(n_gram_model.log_set))
print(len(n_gram_model.uncertain_log))
print(len(n_gram_model.ground_truth_log))

150370
150370
9166
9166


In [ ]:
n_gram_model.eval_model(n=2)

In [ ]:
for n in range(2,5):
    print('Running experiment on N =', n)
    n_gram_model.eval_model(n)
    print('\n\n')

### Weak Order Model
---

estimates the probabilities determining the fraction of traces an event related to some activity $a$ occurs at some point before another activity $b$ <br>
formally the predicate $order$ describes whether or not a trace $\sigma = \langle E_1, ..., E_n \rangle$ contains two activities $a$ and $b$ in weak order:
$$order(a, b, \sigma) \iff \exists \; i,j : \;\;\; e_i \in E_i \land e_j \in E_j \land \lambda(e_i) = a \land \lambda(e_j) = b \land i < j \;\;\;\; i,j \in \{1, ..., n\}$$
$order$ allows to estimate a probability for specific activity executions in weak order by considering the ratio of traces that contain the respective events:
$$P(a, b) = \frac{\vert \{ \sigma \in \Sigma \;\; \vert \;\; order(a, b, \sigma) \} \vert}{\vert \{ \sigma \in \Sigma \;\; \vert \;\; \exists e, e' \in E_\sigma : \; \lambda(e) = a \land \lambda(e') = b \} \vert}$$
with that the probability of a possible resolution $\phi = \langle e_1, ..., e_n \rangle$ is the computed by aggregating the probabilities of all pairs of events to occur in the particular order:
$$P_{WO}(\phi) = \prod_{\substack{1 \leq i < n \\ i < j \leq n}} P(\lambda(e_i), \lambda(e_j))$$

In [126]:
class WeakOrderModel():

    def __init__(self, log):
        self.NAME = "concept:name"
        self.TIME = "time:timestamp"
        
        self.log = log
        self.log_set = self.__make_log_set()  # log set is a list of traces, where each trace is a dict of (timestamps, events) key-value-pairs
        
        self.uncertain_log, self.ground_truth_log = self.__split_and_sparse_log(log)
    
    
    def __make_log_set(self) -> list:
        log_set = []
        for trace in self.log:
            log_set.append(self.__make_trace_sets(trace))
        return log_set

    
    def __make_trace_sets(self, trace) -> dict:
        trace_set = dict()
        for event in trace:
            trace_set[str(event[self.TIME])] = trace_set.get(str(event[self.TIME]), []) + [event[self.NAME]]
        return trace_set
    
    
    def __split_and_sparse_log(self, log):
        """Prepare the data for evaluation, i.e. the uncertain log and corresponding ground truth."""
        
        uncertain_log, ground_truth_log = [], []
        for trace in log:
            if not self.__trace_is_certain(trace):
                sparse_trace_set = self.__get_sparse_trace_set(trace)
                uncertain_log.append(sparse_trace_set)

                sparse_trace = self.__get_sparse_trace(trace)  # get the ground truth according to gold standard
                ground_truth_log.append(sparse_trace)          # i.e. order in the log

        return uncertain_log, ground_truth_log
    
    
    def __get_sparse_trace(self, trace):
        """ Make a list of activities from the pm4py trace object. """
        sparse_trace = [event[self.NAME] for event in trace]
        return sparse_trace
    
    
    def __get_sparse_trace_set(self, trace) -> dict:
        trace_set = dict()
        for event in trace:
            trace_set[str(event[self.TIME])] = trace_set.get(str(event[self.TIME]), []) + [event[self.NAME]]
  
        trace_set = [event_set for event_set in trace_set.values()]

        return trace_set
    
    
    def __trace_is_certain(self, trace):
        """ Check if a trace is certain."""
        for i in range(len(trace)-1):
            if trace[i][self.TIME] == trace[i+1][self.TIME]:
                return False
        return True
    
    
    def order(self, a: str, b: str, trace_set) -> bool:
        """Check whether a trace contains activity a before activity b."""
        
        trace_list = [event_set for timestamp, event_set in trace_set.items()]
        for i in range(len(trace_list)-1):
            if a in trace_list[i]:
                for j in range(i+1, len(trace_list)):
                    if b in trace_list[j]:
                        return True
        return False

    
    def contains_activities(self, a: str, b: str, trace_set) -> bool:
        """Check whether a trace contains the activities a and b."""
        
        activities = [a for event_set in trace_set.values() for a in event_set]
        return (a in activities and b in activities)


    def P_a_b(self, a: str, b: str) -> float:
        """Computes the probability of activity a and b to occur in weak order."""
        
        n_traces_with_a_b_in_weak_order = 0
        n_traces_with_a_b = 0
        
        for trace_set in self.log_set:
            if self.contains_activities(a, b, trace_set):
                n_traces_with_a_b += 1
                if self.order(a, b, trace_set):
                    n_traces_with_a_b_in_weak_order += 1
        
        if n_traces_with_a_b == 0:
            return 0
        else:
            return n_traces_with_a_b_in_weak_order / n_traces_with_a_b

    
    def P_weak_order(self, pos_res: list) -> float:
        """Compute the probability of a possible resolution accorind to the weak order model."""
        
        result = 1.0
        
        for i in range(len(pos_res)-1):
            for j in range(i+1, len(pos_res)):
                result *= self.P_a_b(pos_res[i], pos_res[j])
        
        return result
    
    
    def eval_model(self):
        correct_pred = 0
        total_pred = len(self.ground_truth_log)  # bpic14: 25502
        
        for i in range(total_pred):
            if i%1 == 0: print(i)
            correct_pred += self.eval_uncertain_trace(self.uncertain_log[i], self.ground_truth_log[i], i)
        
        print()
        print('Accuracy:', correct_pred / total_pred )
                       
        
    def eval_uncertain_trace(self, unc_trace, truth_trace, i):
        """ Determine the most probable possible resolution for a given uncertain trace. """
        
        pos_resolutions = pos_res_for_unc_trace(unc_trace)  
        probs = [(pos_res, self.P_weak_order(pos_res)) for pos_res in pos_resolutions]  # get tuple (prob, pos_res) for each pos_res
        probs.sort(key=lambda tup: tup[1], reverse=True)
        
        pred_trace = probs[0][0]
        prob = probs[0][1]
        
        if i % 100 == 0:
            print(i, unc_trace, (pred_trace, prob), truth_trace)
        
        return pred_trace == truth_trace

In [127]:
weak_order_model = WeakOrderModel(event_log)

In [128]:
print(len(weak_order_model.log))
print(len(weak_order_model.uncertain_log))
print(len(weak_order_model.ground_truth_log))

1000
494
494


In [129]:
weak_order_model.eval_model()

0
0 [['tau split8'], ['a10', 'tau join9'], ['g12'], ['c13'], ['e14'], ['f15', 'd16']] (['tau split8', 'a10', 'tau join9', 'g12', 'c13', 'e14', 'f15', 'd16'], 0.019361725780867273) ['tau split8', 'a10', 'tau join9', 'g12', 'c13', 'e14', 'f15', 'd16']
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
100 [['a2'], ['h3'], ['tau join1'], ['g4', 'tau split0', 'tau split0'], ['b6'], ['d7'], ['c5']] (['a2', 'h3', 'tau join1', 'g4', 'tau split0', 'tau split0', 'b6', 'd7', 'c5'], 0.0) ['a2', 'h3', 'tau join1', 'g4', 'b6', 'd7', 'tau split0', 'tau split0', 'c5']
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150